In [19]:
import pandas as pd
import boto3
from io import StringIO

In [15]:
pd.set_option('display.max_columns', None)

In [43]:
client = boto3.client('s3', aws_access_key_id='', aws_secret_access_key='')

In [23]:
bucket = 'bsa-correlation-one'
object_key = 'electronics/cellphone_orders.csv'

csv_obj = client.get_object(Bucket=bucket, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('utf-8')

orders = pd.read_csv(StringIO(csv_string), dtype={'seller_item_gtin':'str'})

In [24]:
orders.head()

,olist_order_id,gmv,revenue,cogs,gross_profit,olist_order_created_at,olist_order_channel_order_code,channel_slug,channel_store,olist_order_purchase_timestamp,olist_order_status,olist_order_approved_at,olist_order_invoice_issue_date,olist_order_invoice_key,olist_order_invoice_number,olist_order_invoice_serial_number,olist_order_last_update,olist_order_last_update_timestamp,olist_order_estimated_delivery_date,olist_order_estimated_delivery_shift,olist_order_delivered_customer_date,olist_order_total_discount,olist_order_total_interest,invoice_url,branded_store_slug,unavailable_reason,olist_item_id,olist_item_sku,olist_item_full_name,olist_item_quantity,olist_item_currency,olist_item_price,carrier,olist_item_freight_value,olist_item_freight_mode,olist_shipment_estimated_delivery,olist_shipment_delivered_carrier_date,olist_shipment_carrier_name,olist_shipping_calculation_type,olist_shipping_shipping_method_display_name,seller_id,seller_order_id,seller_order_created_at,seller_order_updated_at,shipping_limit_date,seller_order_code,seller_order_purchase_timestamp,seller_order_status,seller_order_approved_at,seller_order_availability_days,seller_order_delivered_customer_date,seller_order_seller_brand,seller_order_invoice_danfe_url,seller_order_cancelation_reason,seller_order_cancelation_status,seller_order_suspension_reason,seller_order_invoice_status,seller_order_invoice_id,seller_order_shipping_quote_group_id,seller_item_id,seller_item_product_sku,seller_item_currency,seller_item_price,seller_item_seller_product_code,seller_item_gtin,seller_item_product_attributes,seller_item_availability_days,seller_item_freight_value,seller_item_freight_mode,seller_item_commission_freight_olist,seller_item_commission_product_olist,seller_item_code,seller_item_status,seller_item_price_freight_shift,seller_item_price_discount,seller_item_enable_subsidy
0,d9af5dc5-6b24-487a-8c69-743bfae5f3f0,190.58,36.21,21.92,14.29,2020-01-29 17:07:51,Lojas Americanas-273163198601,b2w,lojasamericanas,2020-01-29 14:52:47,delivered,2020-01-29 17:07:24,2020-01-30 11:40:36,NFe35200109495031000135550010000035531151048592,3553,1,2020-01-28 22:00:00,2020-01-29 16:34:06,2020-03-01 21:00:00,not_scheduled,2020-02-18 12:16:32,7.0,0.0,https://olist-invoice.s3.amazonaws.com/seller/...,olist,NaN,75369d4d-171f-4f9f-9b8c-f47373d8018a,PRDE42DDRGHCC8WN,Celular Dl Yc120 Gsm Dual Chip Azul Entrada An...,1,BRL,132.99,standard,57.59,Normal,2020-03-01 21:00:00,2020-01-31 18:09:38,JadLog,marketplacev2,Normal,73c298c6-3e65-4e1d-bda2-e81623a9ffe6,d147a4bf-66da-49b6-889b-7d7023959ab2,2020-01-29 17:08:00,2020-02-18 12:16:32,2020-02-07 00:59:59,ORDXTFSPE9G9RS1V,2020-01-29 14:52:47,delivered,2020-01-29 17:07:24,0,2020-02-18 12:16:32,Conectado Shop,NaN,NaN,NaN,NaN,invoiced,40e598fd-df17-4aad-adce-fd9899f9066a,f0dfb643-899e-4363-b77b-fa939268c6fd,16a5df86-da88-4d95-b23f-14b0262e1bea,PRDHF5OY4RYRYQN8,BRL,132.99,185,7898567773085,[],0,57.59,Normal,0.19,0.19,IT001ORDXTFSPE9G9RS1V,delivered,0.0,NaN,False
1,943a16b7-5f21-4bc3-8a72-7a3e56269f4f,1027.03,184.87,109.89,74.98,2019-03-07 09:53:23,1955183860,mercadolivre,mercadolivre,2019-03-07 09:53:16,delivered,2019-03-07 09:53:19,2019-03-07 15:04:51,NFe33190310909753001735550010000322381301518860,32238,1,2019-03-06 21:00:00,2019-03-07 09:53:19,2019-03-20 21:00:00,not_scheduled,2019-03-14 00:33:47,0.0,0.0,https://olist-invoice.s3.amazonaws.com/seller/...,girolar,NaN,66135aaa-a14d-420e-b854-e5549633c8dc,GIRICMKECRLCIKYF,Celular Motorola Motog6 Play Dual Chip Indigo,1,BRL,999.00,correios,28.03,me1,2019-03-20 21:00:00,2019-03-09 06:48:47,Correios,Not informed,Normal,b5760b25-a8bd-4e26-acfe-f1b071263126,f7f3f3ed-eae9-4ccf-9143-319b5a6a4b4d,2019-03-07 09:53:26,2019-03-14 00:33:47,2019-03-14 09:53:19,ORDZGDGGHZ7QPISU,2019-03-07 09:53:16,delivered,2019-03-07 09:53:19,0,2019-03-14 00:33:47,Lojas Giro lar e lazer,NaN,NaN,NaN,NaN,invoiced,d2d4cb1f-f9a0-42b3-b1d4-6eeca2665d8d,NaN,36fb79c7-1bc8-4dd3-85a6-35333a1d8f4a,PRDBD7T105Z7Y3OI,BRL,999.00,9028087,7892597346707,[],

In [25]:
orders = orders[['gmv', 'revenue', 'cogs', 'gross_profit',
       'channel_slug', 'channel_store', 'olist_order_purchase_timestamp',
       'olist_order_status',
       'olist_order_estimated_delivery_date',
       'olist_order_estimated_delivery_shift',
       'olist_order_total_discount',
       'olist_item_sku',
       'olist_item_full_name', 'olist_item_quantity',
       'olist_item_price', 'carrier', 'olist_item_freight_value',
       'olist_item_freight_mode',
       'seller_item_product_sku', 'seller_item_price',
       'seller_item_gtin',
       'seller_item_product_attributes',
       'seller_item_freight_value',
       'seller_item_commission_freight_olist',
       'seller_item_commission_product_olist',
       'seller_item_price_freight_shift',
       'seller_item_price_discount', 'seller_item_enable_subsidy']].copy() 

In [26]:
orders['olist_order_purchase_timestamp'] = pd.to_datetime(orders.olist_order_purchase_timestamp)

In [27]:
orders = orders.groupby([pd.Grouper(key='olist_order_purchase_timestamp', freq='D'),
                             'seller_item_gtin', 'olist_item_full_name', 'seller_item_enable_subsidy']).sum()\
                    .sort_values(by=['seller_item_gtin', 'olist_order_purchase_timestamp']).reset_index()

In [29]:
for column in ['olist_order_total_discount', 'olist_item_price', 'olist_item_freight_value',
       'seller_item_price', 'seller_item_freight_value',
       'seller_item_commission_freight_olist',
       'seller_item_commission_product_olist',
       'seller_item_price_freight_shift', 'seller_item_price_discount']:
    
    orders[column] = orders[column].values / orders['olist_item_quantity'].values

In [30]:
orders.head()

,olist_order_purchase_timestamp,seller_item_gtin,olist_item_full_name,seller_item_enable_subsidy,gmv,revenue,cogs,gross_profit,olist_order_total_discount,olist_item_quantity,olist_item_price,olist_item_freight_value,seller_item_price,seller_item_freight_value,seller_item_commission_freight_olist,seller_item_commission_product_olist,seller_item_price_freight_shift,seller_item_price_discount
0,2019-04-22,0095673865926,Smartphone Sony Xperia XZ2 Compact H8314 Singl...,False,3937.42,787.48,492.18,295.30,0.0,1,3879.90,57.52,3879.90,57.52,0.20,0.20,0.0,0.0
1,2019-03-05,0190198057792,"Iphone 6 32 GB Dourado, Tela 4.7 IOS 8, Câmera...",False,1855.33,371.07,231.92,139.15,0.0,1,1819.10,36.23,1819.10,36.23,0.20,0.20,0.0,0.0
2,2019-03-15,0190198057792,"Iphone 6 32 GB Dourado, Tela 4.7 IOS 8, Câmera...",False,1904.08,380.82,238.01,142.81,0.0,1,1819.10,84.98,1819.10,84.98,0.20,0.20,0.0,0.0
3,2019-03-18,0190198057792,"Iphone 6 32 GB Dourado, Tela 4.7 IOS 8, Câmera...",False,1852.04,370.41,231.51,138.90,0.0,1,1819.10,32.94,1819.10,32.94,0.20,0.20,0.0,0.0
4,2019-07-07,0190198058195,"iPhone 6s 32gb Rosê Tela Retina Hd 4,7 3d Tou...",False,2040.00,387.60,220.00,167.60,0.0,1,1999.97,40.03,1999.97,40.03,0.19,0.19,0.0,0.0


In [32]:
orders.shape

(7981, 18)

### Competition

In [35]:
bucket = 'bsa-correlation-one'
object_key = 'electronics/competition_electronics.csv'

csv_obj = client.get_object(Bucket=bucket, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('utf-8')

competition = pd.read_csv(StringIO(csv_string), dtype={'gtin':'str'})

In [36]:
competition['updated_at'] = pd.to_datetime(competition.updated_at)

In [38]:
competition.drop(columns=['Unnamed: 0'], inplace=True)

### History

In [41]:
bucket = 'bsa-correlation-one'
object_key = 'electronics/cellphone_history.csv'

csv_obj = client.get_object(Bucket=bucket, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('utf-8')

history = pd.read_csv(StringIO(csv_string), lineterminator='\n')

In [42]:
history.head()

,availability_days,brand,branded_store_slug,canonical_sku,catalog_feed_date,catalog_feed_id,category,category_info,channel_slug,commission_plan,created_at,currency,description,external_id,group_id,gtin,id,name,offer,offer_discount,parent_id,part_number,partition_0,price,price_freight_shift,reject_reason,seller_product_sku,sent_error_reason,status,stock,updated_at
0,0,LG,olist,PROA5C80A733E03,NaN,NaN,NaN,NaN,cnova,default,2018-05-20 18:01:57,BRL,Marca: LG\r\nModelo: B220\r\nCor: Preto\r\nSIM...,27606136,PROA5C80A733E03,8806084974501,75b51aab-87db-4165-85ae-3f5825f77288,Celular LG B220,104.90,NaN,80b93d28-a453-49f4-b706-e4a05fc1b235,NaN,2018,149.90,0.0,NaN,PRDRMEEHEGXL9C19,NaN,published,82,2018-05-20 18:01:57
1,0,Lenoxx,olist,PRD2O1VYFIAE0SY1,NaN,NaN,18a481e0-db1b-43c7-8313-18d8592213c3,NaN,walmart,default,2019-04-15 16:49:18,BRL,"O celular Flip, CX 908, acabou de ser lançado ...",96273700,PRD2J1LAB56ZYI9A,7897181939082,470ef304-e912-4430-a523-2a7f6d218eb8,Celular Flip com teclas grandes e tecla SOS Le...,185.00,NaN,2cac745b-3c7b-4e9a-acb4-16066665b9b8,NaN,4,229.00,0.0,NaN,PRD2J1LAB56ZYI9A,NaN,published,5,2019-04-15 16:49:18
2,13,Xiaomi,olist,PRDFZYC7OVZSQ7AE,NaN,NaN,1956ea79-35a0-4ad9-9c56-bd63b6b62279,NaN,zoom,default,2019-04-06 12:50:10,BRL,Maior campo de visão\nele tem um display fhd +...,NaN,PRDF26M5M9TFEWDX,6941059610533,c4021ed5-6e96-48e8-8f85-fffc839b8cf7,Smartphone Xiaomi Mi A2 64GB Dual 5.99 20MP V...,1499.90,NaN,f7d874af-614a-4c31-98c4-a0c4543ab4b5,NaN,4,2099.90,0.0,NaN,PRDF26M5M9TFEWDX,NaN,published,5,2019-04-06 12:50:10
3,0,Samsung,girolar,GIRGFCYRT89X6104,NaN,NaN,abba9198-b61e-40f2-957b-ae0266ee47a5,NaN,mercadolivre,default,2019-04-15 13:05:14,BRL,Surpreendase com o novo galaxy j2 core. desenv...,MLB1210765584,PRDN8MOSK8GNJOP3,7892509104586,ea02395e-1274-446a-81f7-2ac48af07acd,Smartphone Samsung Galaxy J2 Core 16GB Prata,579.99,NaN,37db02bb-a140-4b17-94b1-87d10481c8c2,NaN,4,599.99,0.0,NaN,PRDN8MOSK8GNJOP3,NaN,published,15,2019-04-15 13:05:14
4,0,BEDIN SAT,olist,PRDFHVYNQT2N04ZF,NaN,NaN,36cc5e21-b795-4db6-838d-efa3342a15cc,NaN,mercadolivre,reduced,2020-01-02 09:26:09,BRL,O Celular de Mesa 5 bandas BDF-12 Wifi 3G é in...,MLB1123297289,PRD6D2K8LOR8MXT3,7892331000148,352a3b8c-d536-488e-a123-67fb7079512b,Telefone Celular Rural Fixo Mesa 3G WiFi 5 Ba...,461.08,24.27,87531b43-f6f4-4fb8-99f6-7224fbc356e6,NaN,12,485.35,0.0,NaN,PRDXZFANNA4SZD3Y,NaN,published,7,2020-01-02 09:26:09


In [44]:
history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434589 entries, 0 to 434588
Data columns (total 31 columns):
availability_days      434589 non-null int64
brand                  434589 non-null object
branded_store_slug     434589 non-null object
canonical_sku          434589 non-null object
catalog_feed_date      11084 non-null object
catalog_feed_id        11084 non-null float64
category               430796 non-null object
category_info          31296 non-null object
channel_slug           434589 non-null object
commission_plan        434589 non-null object
created_at             434589 non-null object
currency               434589 non-null object
description            434589 non-null object
external_id            312808 non-null object
group_id               418060 non-null object
gtin                   434589 non-null int64
id                     434589 non-null object
name                   434589 non-null object
offer                  434589 non-null float64
offer_discount    